In [75]:
import math
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
from dataclasses import dataclass
from typing import Union

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch.manual_seed(1337)

eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 50 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4

win_size = 5

In [76]:
@dataclass
class ModelArgs:
    d_model: int
    n_layer: int
    vocab_size: int
    d_state: int = 16
    expand: int = 2
    dt_rank: Union[int, str] = 'auto'
    d_conv: int = 4 
    pad_vocab_size_multiple: int = 8
    conv_bias: bool = True
    bias: bool = False
    scan_mode: str = 'cumsum'
    
    def __post_init__(self):
        self.d_inner = int(self.expand * self.d_model)
        
        if self.dt_rank == 'auto':
            self.dt_rank = math.ceil(self.d_model / 16)
            
        if self.vocab_size % self.pad_vocab_size_multiple != 0:
            self.vocab_size += (self.pad_vocab_size_multiple
                                - self.vocab_size % self.pad_vocab_size_multiple)

In [77]:
def select_scan(x, dt, A, B, C, D):
    dA = torch.einsum('bld,dn->bldn', dt, A)
    dB_x =  torch.einsum('bld,bld,bln->bldn', dt, x, B) 

    #This is wrong
    h_t = dA + dB_x
    y = torch.einsum('bldn,bln->bld', h_t, C)
    return y + x*D

def real_scan(x, dt, A, B, C, D):
    dA = torch.einsum('bld,dn->bldn', dt, A)
    dB_x =  torch.einsum('bld,bld,bln->bldn', dt, x, B) 

    dA_sum = F.pad(dA[:, 1:], (0, 0, 0, 0, 0, 1)).flip(1).cumsum(1).exp().flip(1)
    x_o = dB_x * dA_sum
    x_o = x_o.cumsum(1) / (dA_sum * 1e-12)
    y = torch.einsum('bldn,bln->bld', x_o, C)

    return y + x * D

    

In [78]:
class mambaBlock(nn.Module):
    def __init__(self, args):
        super().__init__()
        self.args = args
        self.in_prog = nn.Linear(args.d_model,args.d_inner *2)#split later
        self.conv = nn.Conv1d( in_channels=args.d_inner, out_channels=args.d_inner, bias=args.conv_bias, kernel_size=args.d_conv, groups=args.d_inner, padding=args.d_conv-1)

        self.x_proj = nn.Linear(args.d_inner, args.dt_rank + args.d_state*2, bias=False)
        self.dt_proj = nn.Linear(args.dt_rank, args.d_inner, bias=True)
        
        self.out_proj = nn.Linear(args.d_inner, args.d_model, bias=args.bias)

        A = torch.arange(1, args.d_state+1).repeat(args.d_inner,1)
        #A = torch.repeat(torch.arange(1, n_ssm+1), 'n -> d n', d=hid_dims)
        self.A_log = nn.Parameter(torch.log(A))
        self.D = nn.Parameter(torch.ones(args.d_inner))

    def forward(self, x):
        b, t, e = x.shape

        inp = self.in_prog(x)
        (x, residual) = inp.split(split_size=[self.args.d_inner, self.args.d_inner], dim=-1)
        x = torch.transpose(x, 1, 2)
        x = self.conv(x)[:, :, :t]
        x = torch.transpose(x, 1, 2)

        x = F.silu(x)

        y = self.ssm(x)
        y = y * F.silu(residual)
        out = self.out_proj(y)

        return out

    def ssm(self,x):
        (d_in, n) = self.A_log.shape

        A = -torch.exp(self.A_log.float())
        D = self.D.float()

        x_dbl = self.x_proj(x)
        (delta, B, C) = x_dbl.split(split_size=[self.args.dt_rank, n, n], dim=-1)
        delta = F.softplus(self.dt_proj(delta))
        y = select_scan(x, delta, A, B, C, D)
        return y




In [79]:
args = ModelArgs(64,1,50)
mamba_test = mambaBlock(args)

v = torch.rand(1,512,64)
mamba_test(v)

tensor([[[ 0.1414,  0.1323, -0.0507,  ...,  0.1158, -0.7033,  0.6304],
         [-0.1681,  0.0993, -0.3100,  ...,  0.2998, -0.7032,  0.4442],
         [-0.3408,  0.0081, -0.2153,  ...,  0.1907, -0.7562,  0.0106],
         ...,
         [ 0.1052,  0.0812, -0.0160,  ...,  0.1605, -0.2631,  0.1341],
         [-0.3209,  0.2885, -0.1412,  ...,  0.4429, -1.1145,  0.1711],
         [ 0.0447, -0.0341, -0.1213,  ...,  0.0699, -0.3883,  0.2298]]],
       grad_fn=<UnsafeViewBackward0>)

In [80]:

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

torch.manual_seed(1337)

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]


# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss(model):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits = model(X)
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = Y.view(B*T)
            loss = F.cross_entropy(logits, targets)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [81]:
class RMSNorm(nn.Module):
    def __init__(self, d_model, eps=1e-5):
        super().__init__()
        self.eps = eps
        self.weight = nn.Parameter(torch.ones(d_model))

    def forward(self, x):
        out = x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + self.eps) * self.weight
        return out

In [82]:
class residualBlock(nn.Module):
    def __init__(self, args):
        super().__init__()
        self.args = args
        self.mix = mambaBlock(args)
        self.norm = RMSNorm(args.d_model)

    def forward(self, x):

        out = self.mix(self.norm(x)) + x
        return out

In [83]:
args = ModelArgs(64,1,64)
res = residualBlock(args)

v = torch.rand(1,512,64)
res(v)

tensor([[[ 0.9951,  0.3153,  1.0366,  ...,  1.0954, -1.0815,  1.4929],
         [ 0.1034,  0.4176, -0.2403,  ...,  1.1516, -1.1137,  1.9249],
         [-0.9033, -0.0061,  0.3102,  ...,  0.5350, -1.5517,  0.8793],
         ...,
         [ 0.5716,  0.6790,  0.4099,  ...,  0.8244, -0.1492,  1.1304],
         [-0.2747,  1.3352, -0.0695,  ...,  1.0090, -1.7585,  1.2959],
         [ 0.2788,  0.5609,  0.5027,  ...,  0.9255,  0.7978,  0.5610]]],
       grad_fn=<AddBackward0>)

In [84]:
class Mamba(nn.Module):
    def __init__(self, args):
        super().__init__()
        self.args = args

        self.emb = nn.Embedding(args.vocab_size, args.d_model)
        self.layers = nn.ModuleList([residualBlock(args) for _ in range(args.n_layer)])
        self.norm_f = RMSNorm(args.d_model)

        self.lm_head = nn.Linear(args.d_model, args.vocab_size, bias=False)
        self.lm_head.weight = self.emb.weight #Weight tying paper?

    def forward(self, input):
        x = self.emb(input)

        for layer in self.layers:
            x = layer(x)

        x = self.norm_f(x)
        return self.lm_head(x)
    
    def generate(self, x, max_length):
        #x is a BxT array of in current context
        for _ in range(max_length):
            x_cond = x
            logits = self(x_cond)
            logits = logits[:,-1,:]
            probs = F.softmax(logits, dim=-1)
            x_next = torch.multinomial(probs, num_samples=1)
            x = torch.cat((x, x_next), dim=1)

        return x

In [85]:
args = ModelArgs(64,2,vocab_size)
mambat = Mamba(args)

v = torch.rand(1,512).to(torch.long)
mambat(v)

tensor([[[51.9984,  2.7704, -9.5864,  ..., -7.2773,  7.5230, -2.0269],
         [44.3036,  1.4004, -6.9185,  ..., -3.5175,  8.2403, -2.0923],
         [43.1806,  2.5789, -8.4191,  ..., -2.0741,  6.2041, -3.0936],
         ...,
         [44.4853,  1.8676, -7.7587,  ..., -2.9403,  8.1310, -2.1525],
         [44.4853,  1.8676, -7.7587,  ..., -2.9403,  8.1310, -2.1525],
         [44.4853,  1.8676, -7.7587,  ..., -2.9403,  8.1310, -2.1525]]],
       grad_fn=<UnsafeViewBackward0>)

In [86]:
model = Mamba(args
    )
m = model.to(device)
# print the number of parameters in the model
#print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')
print(next(m.parameters()).is_cuda)
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss(model)
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits = model(xb)
    B, T, C = logits.shape
    logits = logits.view(B*T, C)
    targets = yb.view(B*T)
    loss = F.cross_entropy(logits, targets)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

context = torch.zeros((1, 50), dtype=torch.long, device=device)
print(decode(m.generate(context, max_length=500)[0].tolist()))

True
step 0: train loss 53.3138, val loss 53.2302
step 500: train loss 2.4426, val loss 2.4919
step 1000: train loss 2.2177, val loss 2.2666
step 1500: train loss 2.0919, val loss 2.1646
step 2000: train loss 2.0077, val loss 2.0898
step 2500: train loss 1.9417, val loss 2.0487
step 3000: train loss 1.8858, val loss 2.0157
step 3500: train loss 1.8414, val loss 1.9848
step 4000: train loss 1.8047, val loss 1.9570
step 4500: train loss 1.7688, val loss 1.9330
step 4999: train loss 1.7391, val loss 1.9083


















































CAMILLORENCE:
O, and is by blood: and boaze we and Our my dalied
My art this usque, to back that a enarice, you, not commonour
You&ge to find come mild did bout miree:
You in latistly; ov tenge.

WARWISSTHARINCENTIO:
Nand me littis my nevery well; and plaw ye will full afore elaves
Morthy would that
And insunch their as worth for is The chiefus poor of his but than nonrurd fir at is his land, I male ofter:
Firse?

KING HENRY VI:
Har,
Sbo

In [88]:
cont = torch.zeros((1, 50), dtype=torch.long, device=device)
print(decode(m.generate(cont, max_length=500)[0].tolist()))



















































MARIANE:
Ayboy heave, I'll Hene; go wall the vary, cor this doy and to were sunglalts be wewe minds and enoursingin it so fit this fastork, acclast a brangel wise.In as tall time all as it.--any in thind lament tell true sir, brow neiter have eato I did;
Save thou for Hore
Than Turl broop!' the oftretest just defer worch; hash, are but brow love is cond juslet you thould qunver mutter.

THENRY VI:
Crial.

Unst,
How fabber I not was him.

RICHARD Twree teny, and light:
He yield!

BRUKET:
May, wel
